In [2]:
# pip install transformers

In [3]:
# pip install torch

In [4]:
# pip install tweetnlp

In [5]:
# pip install emoji

In [6]:
# pip install --upgrade scikit-learn

In [7]:
# pip install textblob

In [8]:
# pip install symspellpy

In [9]:
# pip install spacy

In [10]:
# pip install protobuf==3.20

In [2]:
import pandas as pd
import numpy as np
import seaborn as sb
from matplotlib import pyplot as plt
from datetime import datetime
import string

from transformers import AutoTokenizer, AutoModelForSequenceClassification
from nltk.tokenize import TweetTokenizer
from scipy.special import softmax

import nltk

import re
import contractions


import torch
import os

In [3]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to C:\Users\Krithika
[nltk_data]     JK\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
data = pd.read_csv('replyTweets.csv')

C:\Users\Krithika JK\AppData\Local\Temp\ipykernel_23628\908094907.py:1: DtypeWarning: Columns (6,9) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('replyTweets.csv')


# EDA

In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171006 entries, 0 to 171005
Data columns (total 15 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               171006 non-null  int64  
 1   createdAt        171006 non-null  object 
 2   fullName         170988 non-null  object 
 3   userName         171006 non-null  object 
 4   profileImage     171006 non-null  object 
 5   fullText         171006 non-null  object 
 6   replyTo          154370 non-null  object 
 7   lang             171006 non-null  object 
 8   quoteCount       171006 non-null  int64  
 9   retweetCount     171006 non-null  object 
 10  replyCount       171006 non-null  int64  
 11  likeCount        171006 non-null  int64  
 12  viewCount        120511 non-null  float64
 13  sentimentLabel1  995 non-null     float64
 14  sentimentLabel2  250 non-null     float64
dtypes: float64(3), int64(4), object(8)
memory usage: 19.6+ MB


In [17]:
data.head(3)

,id,createdAt,fullName,userName,profileImage,fullText,replyTo,lang,quoteCount,retweetCount,replyCount,likeCount,viewCount,sentimentLabel1,sentimentLabel2
0,1773239429001355747,2024-03-28 06:43:19+00:00,Stella Patchouli,StellaPatch,https://pbs.twimg.com/profile_images/175397939...,@nytimes The curse on Musk iis working!\r\n🔥☔🐱🐱🤹💥,1772931578835939584.0,en,0,0,0,0,9.0,0.0,1.0
1,1773173585424548296,2024-03-28 02:21:41+00:00,Ceti Alpha V,RichGreenwood5,https://pbs.twimg.com/profile_images/122737855...,@elonmusk and @joebiden walk into a Chinese ba...,NaN,en,0,0,0,0,10.0,2.0,0.0
2,1772931578835939531,2024-03-27 10:20:02+00:00,The New York Times,nytimes,https://pbs.twimg.com/profile_images/109824457...,Elon Musk helped create China’s electric vehic...,NaN,en,5,20,52,98,113250.0,1.0,1.0


In [21]:
data.shape

(171006, 15)

In [6]:
data['lang'].value_counts()

en                                                    169915
tl                                                       316
ht                                                       261
fr                                                       133
es                                                       100
in                                                        37
nl                                                        28
no                                                        25
de                                                        24
sv                                                        24
it                                                        23
et                                                        21
pt                                                        18
tr                                                        16
zxx                                                       13
qme                                                       12
und                     

In [7]:
# Filter only eng tweets
data = data[data['lang']=='en']

In [8]:
data['lang'].value_counts()

en    169915
Name: lang, dtype: int64

# Data Pre-Processing

In [9]:
data['cleaned_tweet'] = data.fullText.str.lower()

In [10]:
#replace https
data.cleaned_tweet = data.cleaned_tweet.apply(lambda x: re.sub(r'https?:\/\/\S+', '', x))
data.cleaned_tweet.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", '', x))

0         @nytimes the curse on musk iis working!\r\n🔥☔🐱🐱🤹💥
1         @elonmusk and @joebiden walk into a chinese ba...
2         elon musk helped create china’s electric vehic...
3         lies.\r\n\r\nmusk faced even greater challenge...
4         @sp4rksaflyin @nytimes these electric tesla ca...
                                ...                        
171001    @wsj what’s more democratic than a referendum ...
171002    @wsj is he inhuman? why he has no right to ope...
171003    @wsj it's hilarious watching elon play you guy...
171004                                          @wsj uh so?
171005    @wsj no, trump is old news! he has to many law...
Name: cleaned_tweet, Length: 169915, dtype: object

In [11]:
data.cleaned_tweet = data.cleaned_tweet.apply(lambda x: re.sub(r'{link}', '', x))
data.cleaned_tweet = data.cleaned_tweet.apply(lambda x: re.sub(r"\[video\]", '', x))

In [12]:
# # Define a function to replace '@username' with '@user' except for '@elonmusk'
# def replace_username(text):
#     # Replace all occurrences of '@username' except '@elonmusk'
#     return re.sub(r'(@\w+)(?!elonmusk)', r'@user', text)

# # Apply the function to the 'tweet_text' column
# data['cleaned_tweet'] = data['cleaned_tweet'].apply(replace_username)


In [13]:
def remove_mentions(text):
    return re.sub(r'@\w+', '', text)
data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_mentions)

In [14]:
# Add LOL definition to contractions

import contractions

# Function to expand contractions
def expand_contractions(text):
    expanded_words = []
    for word in text.split():
        expanded_words.append(contractions.fix(word))
    return ' '.join(expanded_words)

# Apply the function to the 'fullText' column
data['cleaned_tweet'] = data['cleaned_tweet'].apply(expand_contractions)

# Display the original and expanded texts
print("Original Text:")
print(data['fullText'])
print("\nExpanded Text:")
print(data['cleaned_tweet'])


Original Text:
0         @nytimes The curse on Musk iis working!\r\n🔥☔🐱🐱🤹💥
1         @elonmusk and @joebiden walk into a Chinese ba...
2         Elon Musk helped create China’s electric vehic...
3         Lies.\r\n\r\nMusk faced even greater challenge...
4         @Sp4rksaflyin @nytimes These Electric Tesla ca...
                                ...                        
171001    @WSJ What’s more democratic than a referendum ...
171002    @WSJ Is he inhuman? Why he has no right to ope...
171003    @WSJ It's hilarious watching Elon play you guy...
171004                                          @WSJ Uh so?
171005    @WSJ No, Trump is old news! He has to many law...
Name: fullText, Length: 169915, dtype: object

Expanded Text:
0                     the curse on musk iis working! 🔥☔🐱🐱🤹💥
1         and walk into a chinese bar. elon makes money ...
2         elon musk helped create china’s electric vehic...
3         lies. musk faced even greater challenges from ...
4         these electri

In [15]:
import emoji

def convert_emojis_to_text(text):
    # Convert emojis to text
    text_with_emojis = emoji.demojize(text)
    return text_with_emojis

# Example usage
text_with_emojis = "I'm feeling 😊 today!"
text_without_emojis = convert_emojis_to_text(text_with_emojis)

# Apply to data
data['cleaned_tweet'] = data['cleaned_tweet'].apply(convert_emojis_to_text)

print("Text with emojis:", text_with_emojis)
print("Text without emojis:", text_without_emojis)


Text with emojis: I'm feeling 😊 today!
Text without emojis: I'm feeling :smiling_face_with_smiling_eyes: today!


In [16]:
# #remove stop words 

# # from nltk.corpus import stopwords
# # nltk.download('stopwords')
# # nltk.download('punkt')

# def remove_stopwords(text):
#     stop_words = set(stopwords.words('english'))
#     word_tokens = nltk.word_tokenize(text)
#     filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
#     return ' '.join(filtered_text)

# # Apply the remove_stopwords function to the 'cleaned_tweet' column
# data['cleaned_tweet'] = data['cleaned_tweet'].apply(remove_stopwords)

In [17]:
# # Spell check - does not work. It takes too long to run
# sypmspellpy does not work
# def spell_check(tweet):
#     spell = SpellChecker()
#     corrected_tokens = []
#     tokens = tweet.split()  # Assuming each tweet is tokenized already
#     for token in tokens:
#         # Check if the token is misspelled
#         corrected_token = spell.correction(token)
#         if corrected_token is not None:
#             corrected_tokens.append(corrected_token)
#     return ' '.join(corrected_tokens)

# data['cleaned_tweet'] = data['cleaned_tweet'].apply(spell_check)

In [18]:
#non letter
data.cleaned_tweet = data.cleaned_tweet.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", '', x))

In [19]:
# tknzr = TweetTokenizer()
# data['tokens'] = data['cleaned_tweet'].apply(tknzr.tokenize)

In [20]:
lemmatizer = nltk.stem.WordNetLemmatizer()
w_tokenizer = TweetTokenizer()
def lemmatize_text(text):
    return [(lemmatizer.lemmatize(w)) for w in \
                                     w_tokenizer.tokenize((text))]
data['tokens'] = data['cleaned_tweet'].apply(lemmatize_text)


In [21]:
PUNCUATION_LIST = list(string.punctuation)
def remove_punctuation(word_list):
    """Remove punctuation tokens from a list of tokens"""
    return [w for w in word_list if w not in PUNCUATION_LIST]
data['tokens'] = data['tokens'].apply(remove_punctuation)

In [22]:
data.dropna(subset=['cleaned_tweet'], inplace=True)

In [23]:
data[['fullText', 'cleaned_tweet', 'tokens']].head(100)

,fullText,cleaned_tweet,tokens
0,@nytimes The curse on Musk iis working!\r\n🔥☔🐱🐱🤹💥,the curse on musk iis working :fire::umbrellaw...,"[the, curse, on, musk, ii, working, fire, ::, ..."
1,@elonmusk and @joebiden walk into a Chinese ba...,and walk into a chinese bar elon makes money l...,"[and, walk, into, a, chinese, bar, elon, make,..."
2,Elon Musk helped create China’s electric vehic...,elon musk helped create chinas electric vehicl...,"[elon, musk, helped, create, china, electric, ..."
3,Lies.\r\n\r\nMusk faced even greater challenge...,lies musk faced even greater challenges from h...,"[lie, musk, faced, even, greater, challenge, f..."
4,@Sp4rksaflyin @nytimes These Electric Tesla ca...,these electric tesla cars cannot start in the ...,"[these, electric, tesla, car, cannot, start, i..."
...,...,...,...
95,@global13news @guardiannews It’s a pretty repu...,it is a pretty reputable source sensor tower e...,"[it, is, a, pretty, reputable, source, sensor,..."
96,@guardiannews Haha... Off course it has 😋\r\nA...,haha off course it has :facesavoringfood: and ...,"[haha, off, course, it, ha, facesavoringfood, ..."
97,Please share this post with your friends who s...,please share this post with your friends who s...,"[please, share, this, post, with, your, friend..."
98,@guardiannews No. This is erroneous.,no this is erroneous,"[no, this, is, erroneous]"


In [45]:
# from wordcloud import WordCloud
# import matplotlib.pyplot as plt

# # Convert list of token lists to a single list of tokens
# all_tokens = [token for tokens_list in data['tokens'] for token in tokens_list]

# # Create a WordCloud object with a TrueType font specified
# wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(dict(nltk.FreqDist(all_tokens)))

# # Plot the word cloud
# plt.figure(figsize=(10, 5))
# plt.imshow(wordcloud, interpolation='bilinear')
# plt.axis('off')
# plt.show()


In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 169915 entries, 0 to 171005
Data columns (total 17 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               169915 non-null  int64  
 1   createdAt        169915 non-null  object 
 2   fullName         169897 non-null  object 
 3   userName         169915 non-null  object 
 4   profileImage     169915 non-null  object 
 5   fullText         169915 non-null  object 
 6   replyTo          153427 non-null  object 
 7   lang             169915 non-null  object 
 8   quoteCount       169915 non-null  int64  
 9   retweetCount     169915 non-null  object 
 10  replyCount       169915 non-null  int64  
 11  likeCount        169915 non-null  int64  
 12  viewCount        119700 non-null  float64
 13  sentimentLabel1  995 non-null     float64
 14  sentimentLabel2  250 non-null     float64
 15  cleaned_tweet    169915 non-null  object 
 16  tokens           169915 non-null  obje

In [ ]:
# data.to_csv('preprocesssed_data.csv', index=False)

# RoBerta

In [61]:
labeled_data['sentimentLabel1'].value_counts()

1.0    566
0.0    294
2.0    135
Name: sentimentLabel1, dtype: int64

In [62]:
labeled_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 995 entries, 0 to 998
Data columns (total 17 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               995 non-null    int64  
 1   createdAt        995 non-null    object 
 2   fullName         995 non-null    object 
 3   userName         995 non-null    object 
 4   profileImage     995 non-null    object 
 5   fullText         995 non-null    object 
 6   replyTo          794 non-null    object 
 7   lang             995 non-null    object 
 8   quoteCount       995 non-null    int64  
 9   retweetCount     995 non-null    object 
 10  replyCount       995 non-null    int64  
 11  likeCount        995 non-null    int64  
 12  viewCount        995 non-null    float64
 13  sentimentLabel1  995 non-null    float64
 14  sentimentLabel2  250 non-null    float64
 15  cleaned_tweet    995 non-null    object 
 16  tokens           995 non-null    object 
dtypes: float64(3), i

In [63]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, AutoConfig, AdamW

In [64]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, RandomSampler, SequentialSampler
from tqdm import tqdm
from sklearn.metrics import classification_report


# Load your datasets
# Assuming your datasets are 'data' and 'labeled_data' DataFrames

# Split labeled data into train and validation sets
train_data, val_data = train_test_split(labeled_data, test_size=0.2, random_state=42)

# Load pre-trained RoBERTa tokenizer
MODEL = "cardiffnlp/twitter-roberta-base-sentiment-latest"
tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# Tokenize text data
train_encodings = tokenizer(list(train_data['cleaned_tweet']), truncation=True, padding=True, max_length=128, return_tensors='pt')
val_encodings = tokenizer(list(val_data['cleaned_tweet']), truncation=True, padding=True, max_length=128, return_tensors='pt')

# Convert labels to tensors of type torch.long
train_labels = torch.tensor(list(train_data['sentimentLabel1']), dtype=torch.long)
val_labels = torch.tensor(list(val_data['sentimentLabel1']), dtype=torch.long)

# Define TensorDatasets for training and validation
train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
val_dataset = TensorDataset(val_encodings['input_ids'], val_encodings['attention_mask'], val_labels)

# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained(MODEL, config=config)

# Define training parameters
batch_size = 32
epochs = 3
learning_rate = 2e-5

# Define optimizer and loss function
optimizer = AdamW(model.parameters(), lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

# Prepare DataLoader for training and validation
train_loader = DataLoader(train_dataset, batch_size=batch_size, sampler=RandomSampler(train_dataset))
val_loader = DataLoader(val_dataset, batch_size=batch_size, sampler=SequentialSampler(val_dataset))

# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch in tqdm(train_loader, desc=f'Epoch {epoch + 1}', unit='batches'):
        input_ids, attention_mask, labels = batch
        optimizer.zero_grad()
        logits = model(input_ids, attention_mask=attention_mask).logits
        loss = loss_fn(logits, labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
    print(f'Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}')

# Evaluation
model.eval() 
all_preds, all_labels = [], []
with torch.no_grad():
    for batch in tqdm(val_loader, desc='Evaluation', unit='batches'):
        input_ids, attention_mask, labels = batch
        logits = model(input_ids, attention_mask=attention_mask).logits
        preds = torch.argmax(logits, dim=1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Calculate classification report
report = classification_report(all_labels, all_preds, target_names=['Positive', 'Negative', 'Neutral'])
print("Classification Report:")
print(report)


C:\Users\Krithika JK\AppData\Local\Temp\ipykernel_7796\3995215218.py:27: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  train_labels = torch.tensor(list(train_data['sentimentLabel1']), dtype=torch.long)
C:\Users\Krithika JK\AppData\Local\Temp\ipykernel_7796\3995215218.py:28: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  val_labels = torch.tensor(list(val_data['sentimentLabel1']), dtype=torch.long)
Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another 

Epoch 1, Loss: 0.814280799627304


Epoch 2: 100%|████████████████████████████████████████████████████████████████████| 25/25 [29:10<00:00, 70.02s/batches]


Epoch 2, Loss: 0.6303270065784454


Epoch 3: 100%|████████████████████████████████████████████████████████████████████| 25/25 [11:27<00:00, 27.50s/batches]


Epoch 3, Loss: 0.43103381991386414


Evaluation: 100%|███████████████████████████████████████████████████████████████████| 7/7 [00:51<00:00,  7.31s/batches]


Classification Report:
              precision    recall  f1-score   support

    Positive       0.55      0.89      0.68        54
    Negative       0.88      0.69      0.78       120
     Neutral       0.88      0.60      0.71        25

    accuracy                           0.73       199
   macro avg       0.77      0.73      0.72       199
weighted avg       0.79      0.73      0.74       199



In [65]:
# Create a DataFrame with predicted labels and corresponding text
df_predicted = pd.DataFrame({'text': val_data['cleaned_tweet'], 'predicted_sentiment': all_preds})
df_predicted['roberta_predicted_sentiment'] = df_predicted['predicted_sentiment'].map({0: 'Positive', 1: 'Negative', 2: 'Neutral'})

df_predicted.head(5)

# Save the DataFrame to a CSV file
# df_predicted.to_csv("predicted_labels_with_text.csv", index=False)

,text,predicted_sentiment,roberta_predicted_sentiment
924,so you are reduced to these type political hit...,0,Positive
525,basically and teamed up drop your subscriptions,1,Negative
568,don lemon joins erin exclusively to talk about...,1,Negative
658,what a joke of an organization you are cnn you...,0,Positive
634,nobody cares,0,Positive


In [66]:
data = pd.concat([data, df_predicted['predicted_sentiment']], axis=1)
data = pd.concat([data, df_predicted['roberta_predicted_sentiment']], axis=1)

In [67]:
data[data['roberta_predicted_sentiment'].notnull()].head(10)

,id,createdAt,fullName,userName,profileImage,fullText,replyTo,lang,quoteCount,retweetCount,replyCount,likeCount,viewCount,sentimentLabel1,sentimentLabel2,cleaned_tweet,tokens,predicted_sentiment,roberta_predicted_sentiment
10,1772945117516571098,2024-03-27 11:13:50+00:00,Ahmed,Ahmedali7104,https://pbs.twimg.com/profile_images/173922670...,@nytimes Updates,1772931578835939584.0,en,0,0,0,0,270.0,1.0,1.0,updates,[update],1.0,Negative
23,1772932914474360899,2024-03-27 10:25:20+00:00,Dog Fanatic,DogFanatic101,https://abs.twimg.com/sticky/default_profile_i...,@nytimes Wait. I thought that was Elaine from ...,1772931578835939584.0,en,0,0,0,2,210.0,1.0,1.0,wait i thought that was elaine from seinfeld,"[wait, i, thought, that, wa, elaine, from, sei...",1.0,Negative
25,1772932728616255540,2024-03-27 10:24:36+00:00,Julio the guard donkey ✌🏽 🇺🇸🌻,DonkeyJulio,https://pbs.twimg.com/profile_images/154292508...,@nytimes Elon Musk is one of those people who ...,1772931578835939584.0,en,0,0,1,2,29.0,0.0,0.0,elon musk is one of those people who does not ...,"[elon, musk, is, one, of, those, people, who, ...",0.0,Positive
29,1772931981317226740,2024-03-27 10:21:38+00:00,Lisa Robertson,sugarbritches09,https://pbs.twimg.com/profile_images/172317735...,@nytimes You guys hate him because he isn't pa...,1772931578835939584.0,en,0,0,0,3,10.0,2.0,2.0,you guys hate him because he is not part of yo...,"[you, guy, hate, him, because, he, is, not, pa...",0.0,Positive
30,1772931788295352750,2024-03-27 10:20:52+00:00,Fatima Khan,FatimaKhan47025,https://pbs.twimg.com/profile_images/175655059...,@nytimes Hope! The problems will be tackled,1772931578835939584.0,en,0,0,0,5,144.0,2.0,2.0,hope the problems will be tackled,"[hope, the, problem, will, be, tackled]",1.0,Negative
39,1772886510364230026,2024-03-27 07:20:57+00:00,Usagi,Beehave99,https://pbs.twimg.com/profile_images/174571711...,It also means that vehicles built there are a ...,NaN,en,0,0,0,0,50.0,0.0,0.0,it also means that vehicles built there are a ...,"[it, also, mean, that, vehicle, built, there, ...",0.0,Positive
44,1772884337937064142,2024-03-27 07:12:19+00:00,Dumb Alpha,dumbalpha_,https://pbs.twimg.com/profile_images/160610895...,"@nytimes Elon bitching about the NYT in 3,2,1…",1772883760859496960.0,en,0,0,0,3,16.0,0.0,0.0,elon bitching about the nyt in,"[elon, bitching, about, the, nyt, in]",0.0,Positive
55,1773004981504708673,2024-03-27 15:11:43+00:00,Gleb Bahmutov,bahmutov,https://pbs.twimg.com/profile_images/117450474...,@hackettholistic @guardiannews Blue check. Block,1772760403652587520.0,en,0,0,0,0,3.0,0.0,0.0,blue check block,"[blue, check, block]",0.0,Positive
59,1772965291372286204,2024-03-27 12:34:00+00:00,Luis A. Vegas Vicentini,lavvspan,https://pbs.twimg.com/profile_images/177113009...,@guardiannews @nickhedley I actually like the ...,1772682700656349696.0,en,0,0,1,1,25.0,2.0,2.0,i actually like the platform much better than ...,"[i, actually, like, the, platform, much, bette...",2.0,Neutral
60,1772959302291968195,2024-03-27 12:10:12+00:00,Scott,minimeadow,https://pbs.twimg.com/profile_images/122055987...,@GenePavlova @guardiannews There are more fake...,1772717872898609664.0,en,0,0,0,0,6.0,0.0,0.0,there are more fake accounts than ever,"[there, are, more, fake, account, than, ever]",1.0,Negative


In [ ]:
# Definition of the labels
labels_definition = config.id2label
print("Labels Definition:", labels_definition)